<a href="https://colab.research.google.com/github/CuteCat24/sifrimalt/blob/main/laptop_sifrimalt_form_to_shopify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create shopify update

In [33]:
# connecting to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
# gsheet to csv
import gspread 
import pandas as pd 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 


auth.authenticate_user()

gc = gspread.authorize(GoogleCredentials.get_application_default()) 

respone_worksheet = gc.open('סיפרימלט (תגובות)').sheet1
shopify_worksheet = gc.open('products_export').sheet1


# get_all_values gives a list of rows. 
response_rows = respone_worksheet.get_all_values() 
shopify_rows = shopify_worksheet.get_all_values()

# Convert to a DataFrame and render. 
form_response = pd.DataFrame.from_records(data=response_rows[1:], columns=response_rows[0])
form_response_shopify = pd.DataFrame.from_records(data=shopify_rows[1:], columns=shopify_rows[0])


In [ ]:
#import pandas

#form_response = pandas.read_csv('/content/drive/MyDrive/sifrimalt/סיפרימלט (תגובות) - תגובות לטופס 1.csv')

In [ ]:
#form_response_shopify= pandas.read_csv('/content/drive/MyDrive/sifrimalt/products_export - products_export (1).csv')

NameError: ignored

In [3]:
form_response

,חותמת זמן,שם התורם,כיתה,שם הספר,שם הסופר,מצב הספר,לאיזה גיל הספר?,סוג,שפה,צלמו תמונה של הכריכה הקדמית והאחורית של הספר
0,15/10/2021 18:56:15,גליה אברמוביץ',ו'2,נוצות ברוח,שלמה אבס,5,"א, ב",אגדות ומשלים,עברית,https://drive.google.com/open?id=1lKGTCOuzc5B7...
1,30/10/2021 13:41:48,גליה אברמוביץ',ו'2,הנסיכה בשחור,שאנון הייל ודין הייל,4,"א, ב","הרפתקאות, פנטזיה",עברית,https://drive.google.com/open?id=1w8ltRLCyNXnt...
2,30/10/2021 13:45:45,גליה אברמוביץ',ו'2,הנסיכה והקוסם,ג'וליה דונלדסון,2,א,אגדות ומשלים,עברית,https://drive.google.com/open?id=1KP7PHVb1MZtt...
3,30/10/2021 13:48:05,גליה אברמוביץ',ו'2,המכשפה של ליאור,שולה מודן,4,א,"פנטזיה, אגדות ומשלים",עברית,https://drive.google.com/open?id=11C1vGnh2cdzD...
4,30/10/2021 13:52:14,גליה אברמוביץ',ו'2,נשים קטנות,לואיזה מיי אלקוט,5,ה,הרפתקאות,עברית,https://drive.google.com/open?id=1KU4zLkj-IMNO...


In [4]:
form_response_shopify

,Handle,Title,Body (HTML),Vendor,Standard Product Type,Custom Product Type,Tags,Published,Option1 Name,Option1 Value,Option2 Name,Option2 Value,Option3 Name,Option3 Value,Variant SKU,Variant Grams,Variant Inventory Tracker,Variant Inventory Qty,Variant Inventory Policy,Variant Fulfillment Service,Variant Price,Variant Compare At Price,Variant Requires Shipping,Variant Taxable,Variant Barcode,Image Src,Image Position,Image Alt Text,Gift Card,SEO Title,SEO Description,Google Shopping / Google Product Category,Google Shopping / Gender,Google Shopping / Age Group,Google Shopping / MPN,Google Shopping / AdWords Grouping,Google Shopping / AdWords Labels,Google Shopping / Condition,Google Shopping / Custom Product,Google Shopping / Custom Label 0,Google Shopping / Custom Label 1,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item,Status
0,נוצות-ברוח,נוצות ברוח,,סיפרימלט,,,,TRUE,כיתה,א,סופר,שלמה אבס,מצב הספר,כמו חדש,,0,shopify,1,deny,manual,2.9,,TRUE,TRUE,,https://cdn.shopify.com/s/files/1/0604/1862/16...,1,,FALSE,,,,,,,,,,,,,,,,,kg,,,active


שם הספר -> Title	
שם הספר without spaces -> Handle

 (value ) שם הסופר -> Option2 Value	
  שם הסופר -> Option2 Name		


In [17]:
for i, column in enumerate(form_response.columns):
  print(i, column)

0 חותמת זמן
1 שם התורם
2 כיתה
3 שם הספר
4 שם  הסופר
5 מצב הספר
6 לאיזה גיל הספר?
7 סוג
8 שפה
9 צלמו תמונה של הכריכה הקדמית והאחורית של הספר
10 
11 מחיר


In [95]:

tmp_table = form_response[['שם הספר', 'שם  הסופר',
              'לאיזה גיל הספר?', 'מצב הספר',
              'צלמו תמונה של הכריכה הקדמית והאחורית של הספר', 
              'מחיר']]

tmp_table = tmp_table.rename(
    columns=({'שם הספר': 'Title', 'שם  הסופר': 'Option2 Value',
              'לאיזה גיל הספר?':'Option1 Value', 'מצב הספר':'Option3 Value',
              'צלמו תמונה של הכריכה הקדמית והאחורית של הספר':'Image Src', 
              'מחיר':'Variant Price'}), 
    #inplace=True,
)

new_table = pd.concat([form_response_shopify, tmp_table])
new_table['Handle'] = new_table['Title']
# TODO(galia): add comment
new_table['Option3 Value'].replace(to_replace='5', value='כמו חדש', inplace=True)
new_table['Option3 Value'].replace(to_replace='4', value='כמו חדש', inplace=True)
new_table['Option3 Value'].replace(to_replace='3', value='בסדר', inplace=True)
new_table['Option3 Value'].replace(to_replace='2', value='משומש', inplace=True)
new_table['Option3 Value'].replace(to_replace='1', value='משומש מאוד', inplace=True)

# TODO(galia): add comment
new_table['Image Src'] = new_table['Image Src'].str.split(pat=", ", expand=True)[0]
new_table['Image Src'] = new_table['Image Src'].str.replace(pat='https://drive.google.com/open\?id', repl='https://docs.google.com/uc?export=download&confirm=no_antivirus&id')

# TODO(galia): add comment
new_table['Option1 Name'] = 'כיתה'
new_table['Option2 Name'] = 'סופר'
new_table['Option3 Name'] = 'מצב הספר'
new_table['Vendor']='סיפרימלט'
new_table['Status'] = 'active'
new_table['Variant Inventory Policy'] = 'deny'
new_table['Variant Fulfillment Service'] = 'manual'
new_table['Gift Card'] = False
new_table['Variant Inventory Qty'] = 1
new_table['Variant Inventory Tracker'] = 'shopify'

new_table

,Handle,Title,Body (HTML),Vendor,Standard Product Type,Custom Product Type,Tags,Published,Option1 Name,Option1 Value,Option2 Name,Option2 Value,Option3 Name,Option3 Value,Variant SKU,Variant Grams,Variant Inventory Tracker,Variant Inventory Qty,Variant Inventory Policy,Variant Fulfillment Service,Variant Price,Variant Compare At Price,Variant Requires Shipping,Variant Taxable,Variant Barcode,Image Src,Image Position,Image Alt Text,Gift Card,SEO Title,SEO Description,Google Shopping / Google Product Category,Google Shopping / Gender,Google Shopping / Age Group,Google Shopping / MPN,Google Shopping / AdWords Grouping,Google Shopping / AdWords Labels,Google Shopping / Condition,Google Shopping / Custom Product,Google Shopping / Custom Label 0,Google Shopping / Custom Label 1,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item,Status
0,נוצות ברוח,נוצות ברוח,,סיפרימלט,,,,TRUE,כיתה,א,סופר,שלמה אבס,מצב הספר,כמו חדש,,0,shopify,1,deny,manual,2.9,,TRUE,TRUE,,https://cdn.shopify.com/s/files/1/0604/1862/16...,1,,False,,,,,,,,,,,,,,,,,kg,,,active
0,נוצות ברוח,נוצות ברוח,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,"א, ב",סופר,שלמה אבס,מצב הספר,כמו חדש,NaN,NaN,shopify,1,deny,manual,,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active
1,הנסיכה בשחור,הנסיכה בשחור,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,"א, ב",סופר,שאנון הייל ודין הייל,מצב הספר,כמו חדש,NaN,NaN,shopify,1,deny,manual,3,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active
2,הנסיכה והקוסם,הנסיכה והקוסם,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,א,סופר,ג'וליה דונלדסון,מצב הספר,משומש,NaN,NaN,shopify,1,deny,manual,10,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active
3,המכשפה של ליאור,המכשפה של ליאור,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,א,סופר,שולה מודן,מצב הספר,כמו חדש,NaN,NaN,shopify,1,deny,manual,10,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active
4,נשים קטנות,נשים קטנות,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,ה,סופר,לואיזה מיי אלקוט,מצב הספר,כמו חדש,NaN,NaN,shopify,1,deny,manual,5,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active


In [96]:
new_table = new_table[new_table['Variant Price'] != ""]
new_table = new_table[new_table['Handle']!='נוצות ברוח']
#new_table = new_table.drop(['Variant Inventory Qty'], axis=1)


In [97]:
new_table

,Handle,Title,Body (HTML),Vendor,Standard Product Type,Custom Product Type,Tags,Published,Option1 Name,Option1 Value,Option2 Name,Option2 Value,Option3 Name,Option3 Value,Variant SKU,Variant Grams,Variant Inventory Tracker,Variant Inventory Qty,Variant Inventory Policy,Variant Fulfillment Service,Variant Price,Variant Compare At Price,Variant Requires Shipping,Variant Taxable,Variant Barcode,Image Src,Image Position,Image Alt Text,Gift Card,SEO Title,SEO Description,Google Shopping / Google Product Category,Google Shopping / Gender,Google Shopping / Age Group,Google Shopping / MPN,Google Shopping / AdWords Grouping,Google Shopping / AdWords Labels,Google Shopping / Condition,Google Shopping / Custom Product,Google Shopping / Custom Label 0,Google Shopping / Custom Label 1,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item,Status
1,הנסיכה בשחור,הנסיכה בשחור,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,"א, ב",סופר,שאנון הייל ודין הייל,מצב הספר,כמו חדש,NaN,NaN,shopify,1,deny,manual,3,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active
2,הנסיכה והקוסם,הנסיכה והקוסם,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,א,סופר,ג'וליה דונלדסון,מצב הספר,משומש,NaN,NaN,shopify,1,deny,manual,10,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active
3,המכשפה של ליאור,המכשפה של ליאור,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,א,סופר,שולה מודן,מצב הספר,כמו חדש,NaN,NaN,shopify,1,deny,manual,10,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active
4,נשים קטנות,נשים קטנות,NaN,סיפרימלט,NaN,NaN,NaN,NaN,כיתה,ה,סופר,לואיזה מיי אלקוט,מצב הספר,כמו חדש,NaN,NaN,shopify,1,deny,manual,5,NaN,NaN,NaN,NaN,https://docs.google.com/uc?export=download&con...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,active


In [98]:
new_table.to_csv('/content/drive/MyDrive/sifrimalt/update_shopify_1.csv')


# Don't run

In [28]:
form_response_shopify.columns

Index(['Handle', 'Title', 'Body (HTML)', 'Vendor', 'Standard Product Type',
       'Custom Product Type', 'Tags', 'Published', 'Option1 Name',
       'Option1 Value', 'Option2 Name', 'Option2 Value', 'Option3 Name',
       'Option3 Value', 'Variant SKU', 'Variant Grams',
       'Variant Inventory Tracker', 'Variant Inventory Qty',
       'Variant Inventory Policy', 'Variant Fulfillment Service',
       'Variant Price', 'Variant Compare At Price',
       'Variant Requires Shipping', 'Variant Taxable', 'Variant Barcode',
       'Image Src', 'Image Position', 'Image Alt Text', 'Gift Card',
       'SEO Title', 'SEO Description',
       'Google Shopping / Google Product Category', 'Google Shopping / Gender',
       'Google Shopping / Age Group', 'Google Shopping / MPN',
       'Google Shopping / AdWords Grouping',
       'Google Shopping / AdWords Labels', 'Google Shopping / Condition',
       'Google Shopping / Custom Product', 'Google Shopping / Custom Label 0',
       'Google Shopping / 

In [20]:
new_table.columns

Index(['חותמת זמן', 'שם התורם', 'Option1 Value', 'Title', 'Option2 Value',
       'Option3 Value', 'לאיזה גיל הספר?', 'סוג', 'שפה', 'Image Src', '',
       'Variant Price', 'Handle', 'Option1 Name', 'Option2 Name',
       'Option3 Name', 'Vendor'],
      dtype='object')

In [80]:
def minus(super_list, copy_list):
  missing_columns = []
  for item in super_list:
    if item not in copy_list:
      missing_columns.append(item)
  return missing_columns

missing_columns = minus(form_response_shopify.columns, tmp_table.columns)

In [81]:
for missing_column in missing_columns:
  print(missing_column, " is:....", form_response_shopify[missing_column][0])
  

Handle  is:.... נוצות-ברוח
Body (HTML)  is:.... 
Vendor  is:.... סיפרימלט
Standard Product Type  is:.... 
Custom Product Type  is:.... 
Tags  is:.... 
Published  is:.... TRUE
Option1 Name  is:.... כיתה
Option2 Name  is:.... סופר
Option3 Name  is:.... מצב הספר
Variant SKU  is:.... 
Variant Grams  is:.... 0
Variant Inventory Tracker  is:.... shopify
Variant Inventory Qty  is:.... 1
Variant Inventory Policy  is:.... deny
Variant Fulfillment Service  is:.... manual
Variant Compare At Price  is:.... 
Variant Requires Shipping  is:.... TRUE
Variant Taxable  is:.... TRUE
Variant Barcode  is:.... 
Image Position  is:.... 1
Image Alt Text  is:.... 
Gift Card  is:.... FALSE
SEO Title  is:.... 
SEO Description  is:.... 
Google Shopping / Google Product Category  is:.... 
Google Shopping / Gender  is:.... 
Google Shopping / Age Group  is:.... 
Google Shopping / MPN  is:.... 
Google Shopping / AdWords Grouping  is:.... 
Google Shopping / AdWords Labels  is:.... 
Google Shopping / Condition  is:....

You want to convert this:

 https://drive.google.com/file/d/(a bunch of letters and numbers)/view?usp=drivesdk

 

To this:

 https://docs.google.com/uc?export=download&confirm=no_antivirus&id=(a bunch of letters and numbers)

In [ ]:
%ls

sample_data/
